In [2]:
import pandas as pd
# Import
from pandarallel import pandarallel

# Initialization
pandarallel.initialize()

import os
import json
# !pip install pywer
import pywer
# !pip install pyjarowinkler
from pyjarowinkler import distance as jwdistance
from tqdm import tqdm

class Const:
    OCR = 'ocr'
    GROUND = 'groundtruth'
    REGION = 'region'
    LINE = 'line'
    SENTENCE = 'sentence'
    FILE = 'filename'
    DATASET = 'dataset_name'
    PREDICTION = 'prediction'
    PROMPT = 'prompt'
    LANGUAGE = 'language'
    NONE = None


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# !pip install pandarallel
# !pip install pywer
# !pip install pyjarowinkler

### Load results

In [4]:
results = []

for root, dirs, files in os.walk(f'data/processed_data/'):
    for file in files:
        if file.endswith(".csv"):
            input_file = os.path.join(root, file)
            df = pd.read_csv(input_file)
            results.append(df)


../data/datasets/ocr/converted/icdar-2019/icdar-2019.jsonl icdar-2019
../data/datasets/ocr/converted/ajmc-primary/ajmc_primary_text.jsonl ajmc-primary
../data/datasets/ocr/converted/overproof/overproof.jsonl overproof
../data/datasets/ocr/converted/ajmc-mixed/ajmc_mixed.jsonl ajmc-mixed
../data/datasets/ocr/converted/impresso/impresso-nzz.jsonl impresso
../data/datasets/ocr/converted/icdar-2017/icdar-2017.jsonl icdar-2017
../data/datasets/htr/converted/htrec/htrec.jsonl htrec
../data/datasets/asr/converted/ina/ina.jsonl ina


In [13]:
results[0].head()

,language,filename,dataset_name,ocr.line,ocr.sentence,ocr.region,groundtruth.line,groundtruth.sentence,groundtruth.region,prediction.prompt,prediction.sentence,prediction.region,model,prompt,prompt_text,type,file
0,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"Charles, par la grace de Dieu, etc.","Charles, par la grace de Dieu, etc. Savoir fai...",None,"Charles, par la grace de Dieu, etc.","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
1,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"Savoir faisons à touz, presens et avenir, à no...","Charles, par la grace de Dieu, etc. Savoir fai...",None,"Savoir faisons à touz, presens et avenir, à no...","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
2,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,Et avant que le dit Amelin peust sur ce remedi...,"Charles, par la grace de Dieu, etc. Savoir fai...",None,et avant que le dit Amelin peust sur ce remedi...,"Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
3,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,linceulz et ii.,"Charles, par la grace de Dieu, etc. Savoir fai...",None,linceulz et n,"Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
4,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"couvertes, et d’ilecques s’en alerent là où bo...","Charles, par la grace de Dieu, etc. Savoir fai...",None,"convertes, et d'ilecques s'en alerent là où bo...","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl


In [ ]:
data = pd.concat(results)

len(data)

In [ ]:
data.model.unique()


In [ ]:
data.prompt.unique()


In [ ]:
data[f"quality-band"].unique()


In [ ]:
data[f'overall-lvenshtein-improvement'].head()

# Sampling

## Sampling for analysis

In [ ]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# dataset_names = data.dataset_name.unique()
# quality_bands = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# sample_list = []
# # Iterate over all unique datasets
# for dataset in tqdm(dataset_names, total = len(dataset_names)):
#     # Get the unique languages for the current dataset
#     languages = data[data['dataset_name'] == dataset]['language'].unique()
#     print(dataset)
#     # Iterate over each unique language
#     for language in languages:
#         print('  --', language)
#         groundtruth_samples = data[(data['dataset_name'] == dataset) 
#                                    & (data['language'] == language)
#                                    & (data['groundtruth.sentence'].str.len() > 10)].drop_duplicates(subset=['groundtruth.line', 'groundtruth.sentence', 'groundtruth.region'])
#         # Limit the groundtruth_samples to three
#         if len(groundtruth_samples) >= 3:
#             groundtruth_samples = groundtruth_samples.sample(3, random_state=1335)

            
#         print(len(groundtruth_samples))             
#         # Iterate over each unique groundtruth samples
#         for idx, gt_sample in groundtruth_samples.iterrows():
#             prompts = data[data['dataset_name'] == dataset]['prompt'].unique()
                           
#             models = data[data['dataset_name'] == dataset]['model'].unique()
                          
#             improvement_bands = data[data['dataset_name'] == dataset]['Improvement Band'].unique()
#             is_few_shot_or_not = data[data['dataset_name'] == dataset]['type'].unique()

#             # Iterate over each unique prompt
#             for prompt in prompts:
#                 print('    -', prompt)
#                 # Iterate over each unique model
#                 for model in models:
# #                     print('     --', model)
#                     # Iterate over each quality band
#                     for band in quality_bands:
# #                         print('        ---', band)
#                         # Iterate over each improvement band
#                         for improvement_band in improvement_bands:
# #                             print('          ----', improvement_band)
#                             for is_few_shot in is_few_shot_or_not:
# #                                 print('-------', is_few_shot)
#                                 subset = data[(data['dataset_name'] == dataset) 
#                                               & (data['language'] == language)
#                                               & (data['prompt'] == prompt)
#                                               & (data['model'] == model)
#                                               & (data['Improvement Band'] == improvement_band)
#                                               & (data['Quality Band'] == band)
#                                               & (data['type'] == is_few_shot)
#                                               & (data['groundtruth.line'] == gt_sample['groundtruth.line'])
#                                               & (data['groundtruth.sentence'] == gt_sample['groundtruth.sentence'])
#                                               & (data['groundtruth.region'] == gt_sample['groundtruth.region'])]
                                          
#                                 # If the subset is not empty, take a sample
#                                 if not subset.empty:
#                                     sample = subset.sample(1, random_state=1, replace=True)
#                                     sample_list.append(sample)
#     #                                 print(sample)
#     #                             else:
#     #                                 print(f"No samples for Dataset: {dataset}, Language: {language}")

                                      
# # Concatenate all the samples into a single DataFrame
# sample_df = pd.concat(sample_list, ignore_index=True)
                                      

### Order columns

In [ ]:
# sample_df = sample_df.drop(['length', 'NbAlignedChar', 'prompt_text', 'File'], axis=1)

In [ ]:

# # 'index', 
# order = ['filename', 'dataset_name', 'model', 'language', 'prompt', 
#          'Overall Levenshtein Improvement', 'Quality Band', 'Improvement Band',
#          'ocr.line', 'groundtruth.line', 'prediction.line', 
#          'line-lev-ocr', 'line-lev-pred', 'line-lev-improvement',
#          'ocr.sentence', 'groundtruth.sentence', 'prediction.sentence', 
#          'sentence-lev-ocr', 'sentence-lev-pred', 'sentence-lev-improvement', 
#          'ocr.region', 'groundtruth.region', 'prediction.region',
#          'region-lev-ocr', 'region-lev-pred', 'region-lev-improvement', 
#          'article_id', 'century', 'Date', 'Type']

# # Reorder the DataFrame
# sample_df = sample_df[order]


### Write sample

In [ ]:
# from datetime import datetime

# # Use today's date for the filename
# today = datetime.now().strftime('%d%B')  # This will format the date as 'DayMonth'

# # Save the DataFrame to a csv file
# sample_df.to_csv(f'ResultsGPTUpdated{today}.csv', index=False)


### Distribution of WER/CER rates for all datasets in the four quality bands, established via Levenshtein similarity.

In [ ]:
data.type.unique()


In [ ]:
data.prompt.unique()


In [ ]:
data['Improvement Band'].unique()


In [ ]:
data.model.unique()


In [ ]:
data = data.loc[data['model'] != 'LLaMA-13B']
data = data.loc[data['model'] != 'LLAMA-65B']
data = data.loc[data['model'] != 'GPT-3-OLD']




In [ ]:
data.model.unique()


In [ ]:
MODELS = [
       'LLAMA-7B', 'LLAMA-2-7B', 
       'BLOOM-560M',  'BLOOM-3B',  'BLOOM-7.1B', 
       'BLOOMZ-560M', 'BLOOMZ-3B', 'BLOOMZ-7.1B', 
       'OPT-350M', 'OPT-6.7B',
       'GPT-2', 'GPT-3', 'GPT-3.5', 'GPT-4']
MODELS

In [ ]:
limited_models = ['LLAMA-7B', 'LLAMA-2-7B', 
       'BLOOM-560M',  'BLOOM-3B',  'BLOOM-7.1B', 
       'BLOOMZ-560M', 'BLOOMZ-3B', 'BLOOMZ-7.1B', 
       'OPT-350M', 'OPT-6.7B',
       'GPT-2']  # Replace these with your list of 'limited' models
open_models = ['GPT-3', 'GPT-3.5', 'GPT-4']  # Replace these with your list of 'open' models

# Create new 'Access' column
data['Access'] = data['model'].apply(lambda x: 'limited' if x in limited_models else ('open' if x in open_models else 'unknown'))


In [ ]:
data.columns

# Plots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.8)
sns.set_style("whitegrid")

# Define OCR noise level bins
bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# Assign OCR noise level labels
labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# Count the number of unique datasets
n_datasets = data.dataset_name.nunique()
dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
                 'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec', 'ina']

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


n_plots = len(dataset_names)
n_plots_per_figure = 4
n_figures = int(np.ceil(n_plots / n_plots_per_figure))

print(n_figures)

for type_of_experiment in ['zero-shot', 'few-shot']:
# for type_of_experiment in ['language-specific']:
    for error_rate in ['lev']:
        
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(30, 15))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                # Compute the mean WER across line, sentence, and region levels
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                      f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the distribution of improvements for each model
                    _ = sns.boxplot(x='model', y=f'Overall Levenshtein Improvement', data=dataset_data, 
                                    ax=axs[i], order=MODELS, hue='prompt', hue_order=prompt_names)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')

                    axs[i].set_ylim([-1, 1.2])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)  # Rotate x-axis labels
                    axs[i].set_xlabel('')  # Remove x-axis label
                    axs[i].set_ylabel('')  # Remove y-axis label
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')


            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Overall Levenshtein Improvement across Datasets, Models, and Prompts ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.)
sns.set_style("whitegrid")

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


for type_of_experiment in ['zero-shot', 'few-shot']:#
    for error_rate in ['lev']:
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(17, 9))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                    
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                
                dataset_data['model'] = pd.Categorical(dataset_data['model'], categories=MODELS, ordered=True)
                
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                      f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the line plot of improvements for each model
                    _ = sns.lineplot(x='model', y=f'Overall Levenshtein Improvement', 
                                     data=dataset_data.sort_values('model'), 
                                    ax=axs[i], hue='prompt', sort=False, linewidth=2)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')
                    axs[i].set_ylim([-1, 1.2])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)
                    axs[i].set_xlabel('Prompt Complexity')  
                    axs[i].set_ylabel('Overall Levenshtein Improvement')  
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')

            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Evolution of Prompt Complexity across Datasets, Models, and Improvements ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1)
sns.set_style("whitegrid")

for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each language and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'language', f'Overall Levenshtein Improvement'])

        for language in data.language.unique():  # Iterate over each unique language
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.language == language) & (data.type == type_of_experiment)]  # Filter data for the current language
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and language, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and language again, this time averaging the averages for each language and prompt
        average_improvements = average_improvements.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(10, 5))  # Create the plot

        try:
            # Plot the line plot of average improvements for each prompt
            _ = sns.lineplot(x='prompt', y=f'Overall Levenshtein Improvement', hue='language', data=average_improvements, 
                             ax=ax, legend='full')  # Add hue='language' to differentiate lines by language
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement ({type_of_experiment})')  # Modify title since it's no longer specific to one language
        ax.set_ylim([-1, 1.2])
        ax.set_xlabel('Prompt Complexity')  
        ax.set_ylabel('Average Overall Levenshtein Improvement')

        plt.tight_layout()
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()



In [ ]:
for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each language and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'language', f'Overall Levenshtein Improvement'])

        for language in data.language.unique():  # Iterate over each unique language
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.language == language) & (data.type == type_of_experiment)]  # Filter data for the current language
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and language, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and language again, this time averaging the averages for each language and prompt
        average_improvements = average_improvements.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(15, 10))  # Create the plot

        try:
            # Plot the KDE plot of average improvements for each prompt
            for language in average_improvements.language.unique():
                _ = sns.kdeplot(average_improvements[average_improvements.language == language][f'Overall Levenshtein Improvement'], 
                                ax=ax, label=language, lw=2.5)  # Increase line width here
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement KDE ({type_of_experiment})')  # Modify title since it's no longer specific to one language
        ax.set_xlabel('Average Overall Levenshtein Improvement')
        ax.set_ylabel('Density')

        plt.tight_layout()
        plt.legend(title='Language', title_fontsize='13', fontsize='12')  # Increase legend fontsize here
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement KDE ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()


In [ ]:
data.columns

In [ ]:
for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each model and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'model', f'Overall Levenshtein Improvement'])

        for model in data.model.unique():  # Iterate over each unique model
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.model == model) & (data.type == type_of_experiment)]  # Filter data for the current model
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and model, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'model'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and model again, this time averaging the averages for each model and prompt
        average_improvements = average_improvements.groupby(['prompt', 'model'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(15, 10))  # Create the plot

        try:
            # Plot the KDE plot of average improvements for each prompt
            for model in average_improvements.model.unique():
                _ = sns.kdeplot(average_improvements[average_improvements.model == model][f'Overall Levenshtein Improvement'], 
                                ax=ax, label=model, lw=2.5)  # Increase line width here
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement KDE ({type_of_experiment})')  # Modify title since it's no longer specific to one model
        ax.set_xlabel('Average Overall Levenshtein Improvement')
        ax.set_ylabel('Density')

        plt.tight_layout()
        plt.legend(title='Model', title_fontsize='13', fontsize='12')  # Increase legend fontsize here
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement KDE ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.5)
sns.set_style("whitegrid")

# Define OCR noise level bins
bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# Assign OCR noise level labels
labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# Count the number of unique datasets
n_datasets = data.dataset_name.nunique()
dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
                 'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec', 'ina']

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


n_plots = len(dataset_names)
n_plots_per_figure = 4
n_figures = int(np.ceil(n_plots / n_plots_per_figure))

for type_of_experiment in ['zero-shot', 'few-shot']:
# for type_of_experiment in ['language-specific']:
    for error_rate in ['lev']:
        
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(30, 15))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                # Compute the mean WER across line, sentence, and region levels
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the distribution of improvements for each model
                    _ = sns.boxplot(x='model', y=f'Overall Levenshtein Improvement', data=dataset_data, 
                                    ax=axs[i], order=MODELS, hue='prompt', hue_order=prompt_names)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')

                    axs[i].set_ylim([-1.1, 1.1])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)  # Rotate x-axis labels
                    axs[i].set_xlabel('')  # Remove x-axis label
                    axs[i].set_ylabel('')  # Remove y-axis label
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')


            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Overall Levenshtein Improvement across Datasets, Models, and Prompts ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=.7)
sns.set_style("whitegrid")

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        fig, axs = plt.subplots(3, 2, figsize=(8, 12))  # Change here
        axs = axs.flatten()  # To make it easy to index

        for i, prompt in enumerate(prompt_names):
            prompt_data = data[(data.prompt == prompt) & (data.type == type_of_experiment)]
            
            if 'icdar' not in dataset:
                prompt_data[f'Overall Levenshtein Improvement'] = prompt_data[[f'line-{error_rate}-improvement', 
                                                                  f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)
            else:
                prompt_data[f'Overall Levenshtein Improvement'] = prompt_data[[f'sentence-{error_rate}-improvement', 
                                                              f'region-{error_rate}-improvement']].mean(axis=1)

            try:
                if len(prompt_data) > 0:
                    sns.kdeplot(data=prompt_data, x=f'Overall Levenshtein Improvement', hue='model', 
                                fill=True, ax=axs[i], hue_order=MODELS)
                    axs[i].set_title(f'{prompt} ({type_of_experiment})')
                    axs[i].set_xlim([-1, 1.2])
                    axs[i].set_ylim([0, 1.4])
            except Exception as ex:
                print(f'Could not plot {prompt} with {ex}')

        # Remove empty subplot
        fig.delaxes(axs[-1])  # Change here

        plt.tight_layout()
        plt.show()
        
# A kernel density estimate (KDE) plot is a method for visualizing the distribution of observations in a dataset, 
# analogous to a histogram. KDE represents the data using a continuous probability density curve in one or more 
# dimensions.

# Relative to a histogram, KDE can produce a plot that is less cluttered and more interpretable, especially when 
# drawing multiple distributions. But it has the potential to introduce distortions if the underlying distribution 
# is bounded or not smooth. Like a histogram, the quality of the representation also depends on the selection of 
# good smoothing parameters.


In [ ]:
data.columns

In [ ]:
prompts_to_compare = ['prompt_complex_02', 'prompt_complex_lang']
languages = [lang for lang in data['language'].unique() if lang != 'en']  # Exclude 'en' language

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        plt.figure(figsize=(10, 6))  # Adjust as necessary

        for language in languages:
            language_data = data[(data.language == language) & 
                                 (data.type == type_of_experiment) & 
                                 (data.prompt.isin(prompts_to_compare))]

            if 'icdar' not in dataset:
                language_data[f'Overall Levenshtein Improvement'] = language_data[[f'line-{error_rate}-improvement', 
                                                                  f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)
            else:
                language_data[f'Overall Levenshtein Improvement'] = language_data[[f'sentence-{error_rate}-improvement', 
                                                              f'region-{error_rate}-improvement']].mean(axis=1)

            try:
                if len(language_data) > 0:
                    sns.kdeplot(data=language_data, x=f'Overall Levenshtein Improvement', hue='prompt', 
                                fill=True, hue_order=prompts_to_compare)
            except Exception as ex:
                print(f'Could not plot {language} with {ex}')

        plt.title(f'All Languages ({type_of_experiment})')
        plt.tight_layout()
        plt.show()


In [ ]:
data.columns

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.)


prompts_to_compare = ['prompt_complex_02', 'prompt_complex_lang']
languages = [lang for lang in data['language'].unique() if lang != 'en']  # Exclude 'en' language

bar_data = []

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        for language in languages:
            for prompt in prompts_to_compare:
                sub_data = data[(data.language == language) & 
                                 (data.type == type_of_experiment) & 
                                 (data.prompt == prompt)]
                if 'icdar' not in dataset:
                    sub_data[f'Overall Levenshtein Improvement'] = sub_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    sub_data[f'Overall Levenshtein Improvement'] = sub_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                
                mean_improvement = np.mean(sub_data[f'Overall Levenshtein Improvement'])
                bar_data.append({'Language': language, 'Prompt': prompt, 'Mean Improvement': mean_improvement})

bar_data = pd.DataFrame(bar_data)

plt.figure(figsize=(10, 6))
sns.barplot(x='Language', y='Mean Improvement', hue='Prompt', data=bar_data, hue_order=prompts_to_compare)
plt.title(f'Mean Levenshtein Improvement for all Languages ({type_of_experiment})')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()


In [ ]:
data.columns

In [ ]:
data.dataset_name.unique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=.7)

segmentations = ['line', 'sentence', 'region']

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        for segmentation in segmentations:
            fig, axs = plt.subplots(3, 2, figsize=(8, 12))  # Change here
            axs = axs.flatten()  # To make it easy to index

            for i, prompt in enumerate(prompt_names):
                prompt_data = data[(data.prompt == prompt) & (data.type == type_of_experiment)]
                if segmentation == 'line':
                    prompt_data = prompt_data[~prompt_data.dataset_name.isin(['icdar-2017', 'icdar-2019'])]
                prompt_data[f'{segmentation.capitalize()} Levenshtein Improvement'] = prompt_data[f'{segmentation}-{error_rate}-improvement']

                try:
                    if len(prompt_data) > 0:
                        sns.kdeplot(data=prompt_data, x=f'{segmentation.capitalize()} Levenshtein Improvement', 
                                    hue='model', fill=True, ax=axs[i], hue_order=MODELS)
                        axs[i].set_title(f'{prompt} ({type_of_experiment})')
                        axs[i].set_xlim([-1, 1.2])
                        axs[i].set_ylim([0, 1.5])
                except Exception as ex:
                    print(f'Could not plot {prompt} with {ex}')

            # Remove empty subplot
            fig.delaxes(axs[-1])  # Change here

            plt.suptitle(f'{segmentation.capitalize()} Levenshtein Improvement across Prompts and Models ({type_of_experiment})', fontsize=20, y=1.02)
            plt.tight_layout()
            plt.show()


In [ ]:
data.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

results = []  # Define results as a list


for type_of_experiment in ['zero-shot', 'few-shot']:
    for model in MODELS:
        model_data = data[(data.model == model) & (data.type == type_of_experiment)]
        
        # Compute the mean Levenshtein Improvement across line, sentence, and region levels
        if 'icdar' not in dataset:
            model_data['Overall Levenshtein Improvement'] = model_data[[f'line-lev-improvement', 
                                                                        f'sentence-lev-improvement', 
                                                                        f'region-lev-improvement']].mean(axis=1)
        else:
            model_data['Overall Levenshtein Improvement'] = model_data[[f'sentence-lev-improvement', 
                                                                        f'region-lev-improvement']].mean(axis=1)

        # Append the results
#         print(model_data['Improvement Band'].unique())
        results.append({'Model': model,
                        'Type of Experiment': type_of_experiment,
                        'Overall Levenshtein Improvement': np.nanmean(model_data['Overall Levenshtein Improvement'])})

# Convert the results list to a DataFrame for plotting
results_df = pd.DataFrame(results)

# Create the plot
plt.figure(figsize=(10,6))
sns.barplot(x='Model', y='Overall Levenshtein Improvement', hue='Type of Experiment', data=results_df, order=MODELS)
plt.title('Overall Levenshtein Improvement for Zero-Shot and Few-Shot Models')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


for type_of_experiment in ['zero-shot', 'few-shot']:
    results = []  # Define results as a list

    for model in MODELS:
        for band in data['Improvement Band'].unique():
            model_band_data = data[(data.model == model) & (data.type == type_of_experiment) & (data['Improvement Band'] == band)]
            
            # Compute the mean Levenshtein Improvement across line, sentence, and region levels
            if 'icdar' not in dataset:
                model_band_data['Overall Levenshtein Improvement'] = model_band_data[[f'line-lev-improvement', 
                                                                                    f'sentence-lev-improvement', 
                                                                                    f'region-lev-improvement']].mean(axis=1)
            else:
                model_band_data['Overall Levenshtein Improvement'] = model_band_data[[f'sentence-lev-improvement', 
                                                                                    f'region-lev-improvement']].mean(axis=1)

            # Append the results
            results.append({'Model': model,
                            'Type of Experiment': type_of_experiment,
                            'Overall Levenshtein Improvement': np.nanmean(model_band_data['Overall Levenshtein Improvement']),
                            'Improvement Band': band})

    # Convert the results list to a DataFrame for plotting
    results_df = pd.DataFrame(results)

    # Create the plot
    plt.figure(figsize=(10,6))
    sns.barplot(x='Model', y='Overall Levenshtein Improvement', hue='Improvement Band', 
                data=results_df, order=MODELS)
    plt.title('Overall Levenshtein Improvement for Zero-Shot and Few-Shot Models')
    plt.show()


In [ ]:
data.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


g = sns.FacetGrid(data, col='Type', hue='Improvement Band', height=10, aspect=1)
g.map(sns.barplot, 'model', 'Overall Levenshtein Improvement', order=MODELS)

# Calculate means for each type of experiment and add horizontal lines
for ax, (type_of_experiment, item) in zip(g.axes.flatten(), data.groupby('Type')):
    mean_improvement = item['Overall Levenshtein Improvement'].mean()
    ax.axhline(mean_improvement, color='black', linestyle='--')
    ax.text(0.6, mean_improvement, f'Mean: {mean_improvement:.2f}', color='black')

plt.show()
